In [1]:
# import plumed
import matplotlib.pyplot as plt
import plumed
import MDAnalysis

/home/yaminb/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
# data=plumed.read_as_pandas("metadNa/dump/dump350.50000.lammpstrj")
import pandas as pd
from pandas import read_csv
#load dataset
df = pd.read_csv('metadNa/dump/dump350.50000.lammpstrj',
                 header=None,delimiter= ' ',usecols=[0,2,3,4], skiprows=9, skipfooter=259)
df

/tmp/ipykernel_21887/787437704.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('metadNa/dump/dump350.50000.lammpstrj',


,0,2,3,4
0,36,0.331316,0.026683,0.160130
1,21,0.228327,0.238400,0.290456
2,226,0.097822,0.447153,0.045229
3,216,0.141983,0.276363,0.154604
4,162,0.186804,0.108334,0.152810
...,...,...,...,...
245,39,0.655349,0.689952,0.912165
246,246,0.624092,0.866215,0.869713
247,114,0.979165,0.981495,0.782731
248,177,0.513536,0.694852,0.832595


In [51]:
df.loc[:,2]

0      0.331316
1      0.228327
2      0.097822
3      0.141983
4      0.186804
         ...   
245    0.655349
246    0.624092
247    0.979165
248    0.513536
249    0.873557
Name: 2, Length: 250, dtype: float64

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
ax.scatter(sequence_containing_x_vals, sequence_containing_y_vals, sequence_containing_z_vals)
plt.show()